In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Currently: 

* $n = 12$
* $k = 8$
* $p \in \{1, 2, 4, 8\}$
* training instances = $15$
* evaluation instances = $500$


Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [9]:
from benchmark.random_k_sat import RandomKSAT

# Read in 100 randomly generated CNF(12, 8, r_8) instances
num_instances = 100
n = 12
k = 8

instances = RandomKSAT.from_poisson(n=n, k=k, instances=num_instances, from_file=True)

In [8]:
from k_sat.pytorch_solver.pytorch_solver import PytorchSolver

f = instances.formulas[10]
ps = PytorchSolver(layers=5)
ps.sat(f, timeout=200)

IndexError: list index out of range

In [25]:
import torch
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit
from k_sat.pytorch_solver.pytorch_optimiser import PytorchOptimiser

ps = [1, 2, 4, 8]#, 30, 60]
optimal_params = {}

for p in ps:
	circuit = PytorchCircuit(num_vars=n, layers=p)
	adam = torch.optim.Adam(circuit.parameters(), lr=0.1, maximize=True)
	optimiser = PytorchOptimiser(circuit, adam, epochs=100)
	optimiser.find_optimal_params(instances[:16])
	optimal_params[p] = (circuit.gamma.detach().clone(), circuit.beta.detach().clone())

Success probability: 0.0004160713288001716
Success probability: 0.005423360969871283
Success probability: 0.000424269528593868
Success probability: 0.01637210138142109
Success probability: 0.0004532740858849138
Success probability: 0.05263092741370201
Success probability: 0.0005650859675370157
Success probability: 0.14667057991027832


Step 2: Evaluate on satisfiable CNF instances

In [ ]:
from statistics import median

k_e = k
n_e = [12]
num_instances_e = 5

# Dictionary results
p_succ = {n : {} for n in n_e}

for n in n_e:
	print(f'evaluating n = {n}')

	# Read in random problem

	instances_e = 

	for (p, params) in optimal_params.items():
		print(f'evaluating p = {p}')
		p_succ_p = 0
		running_times_p = []

		# Evaluate success probability and running time for QAOA
		for formula in instances_e.formulas:
			circuit = encoder.encode_formula(formula, p)
			p_succ_p += evaluator.success_probability(circuit, formula, params)
			ass, running_time_f = evaluator.running_time(circuit, formula, params, timeout=100) 
			if ass == "-1":
				print("QAOA TIMEOUT")
			running_times_p.append(running_time_f)

		# Store results
		p_succ[n][p] = p_succ_p / num_instances_e
		print(running_times_p)
		running_times[n][p] = median(running_times_p)


evaluating n = 12
Walksatlm took 100001000 steps
Walksatlm took 3 steps
Walksatlm took 2 steps
Walksatlm took 100001000 steps
Walksatlm took 8 steps
evaluating p = 1
QAOA TIMEOUT
QAOA TIMEOUT
QAOA TIMEOUT
QAOA TIMEOUT
QAOA TIMEOUT
[1, 1, 1, 1, 1]


In [ ]:
p_succ

{12: {1: 0.017068698906207914}}

In [ ]:
running_times

{12: {1: 1}}